In [3]:
import utils as ut

documents = os.listdir(ut.CORPUS)
doc_path = [ut.CORPUS + '/' + document for document in documents]

### Exemplo 1
PRO
> doesnt depent on orientation \
> doesnt depent on perspective \
> depends only on numpy and cv2

CONS
> Only works well in single documents with well delimited contours

OBS
> can be used for for colored documents in order to eliminate background \
> reorientation idea can be implemented, but in another way

https://dontrepeatyourself.org/post/learn-opencv-by-building-a-document-scanner/ solution

In [6]:
import cv2

def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect
	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))
	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
	# return the warped image
	return warped

def scan_document0(document):
    img = cv2.imread(document, cv2.IMREAD_COLOR)
    plt.imshow(img)
    plt.show()

    height = 800
    width = 600
    green = (0, 255, 0)

    image = cv2.imread(document) # can also read with input == 0 and skip gray
    image = cv2.resize(image, (width, height))
    orig_image = image.copy()

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert the image to gray scale
    blur = cv2.GaussianBlur(gray, (5, 5), 0) # Add Gaussian blur
    edged = cv2.Canny(blur, 75, 200) # Apply the Canny algorithm to find the edges TODO Test other edge alg

    contours, _ = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)


    # plt.imshow(orig_image)

    image_contour = image.copy()
    cv2.drawContours(image_contour, contours, -1, green, 3)
    plt.figure()
    plt.imshow(image_contour)
    plt.show()

    # go through each contour
    for contour in contours:
        # we approximate the contour
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.05 * peri, True)
        # if we found a countour with 4 points we break the for loop
        # (we can assume that we have found our document)
        if len(approx) == 4:
            doc_cnts = approx
            break

    # plt.imshow(orig_image)
    image_contour = image.copy()
    cv2.drawContours(image_contour, [doc_cnts], -1, green, 3)
    plt.figure()
    plt.imshow(image_contour)
    plt.show()

    # plt.imshow(image_contour)
    # plt.show()
    # apply warp perspective to get the top-down view
    warped = four_point_transform(orig_image, doc_cnts.reshape(4, 2))
    # # convert the warped image to grayscale
    # warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)

    plt.imshow(cv2.resize(warped, (600, 800)))
    plt.show()

ModuleNotFoundError: No module named 'cv2'

In [5]:
document = doc_path[4]
scan_document0(document)

NameError: name 'cv2' is not defined